In [1]:
import numpy as np
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.gleu_score import sentence_gleu
from nltk.translate.meteor_score import meteor_score
from google.transliteration import transliterate_text
from indictrans import Transliterator
import re

In [3]:
file_path = "../datasets/HINGE dataset/valid.csv"
df = pd.read_csv(file_path)

In [4]:
df = df[df['Average rating'] >= 5]
df.head()

,English,Hindi,Hinglish,Average rating,Disagreement
0,Are you sure you want to permanently delete th...,क्या आप इन फ़ाइलों को स्थायी रूप से हटाना चाहत...,kya aap in files ko sthayi roop se permanently...,6,4
1,Three public meetings were held in Bombay City...,उस अवसर पर बंबई में तीन सभाएं की गयीं।\n,us avasar par bombay meetings were held ki gay...,7,1
2,Nominee of the insurance has to be a near rela...,बीमा का नामित व्यक्ति अभिदाता का निकट संबंधी ह...,insurance ka namit vyakti abhidata ka nikat sn...,9,2
3,Thus there was an inadequate appreciation of t...,इस प्रकार इस महत्वपूर्ण क्षेत्र में तेजी से का...,is prkar is vital sector inadequate appreciati...,9,0
4,"The Arabs laughed at him , and the alchemist l...",अरब सैनिक उसकी बात सुनकर हंस पड़े । <s> उनके स...,arab sainik uski bat sunakar hns pare arabs s...,8,1


In [5]:
hinglish_sentences = df['Hinglish']
print(hinglish_sentences)

hinglish_sentences = hinglish_sentences.values
print(type(hinglish_sentences))
print(hinglish_sentences.shape)

0      kya aap in files ko sthayi roop se permanently...
1      us avasar par bombay meetings were held ki gay...
2      insurance ka namit vyakti abhidata ka nikat sn...
3      is prkar is vital sector inadequate appreciati...
4      arab sainik uski bat sunakar hns pare arabs  s...
                             ...                        
388    ye hi ve log hain jo tab bhi company ko ek bhi...
389    baharhal , main yah inference nahin lagana cha...
391                             koee interface  s nhin\n
392    antrudak sac ko anta endolymphatic sac ke roop...
394           iska answer satish ke man men ready mind\n
Name: Hinglish, Length: 364, dtype: object
<class 'numpy.ndarray'>
(364,)


In [6]:
hindi_true = list(df['Hindi'])
# print(hindi_true)

In [7]:
# # Testing using LIBINC transliterator
# # trn = Transliterator(source='eng', target='hin', build_lookup=True, decode='beamsearch')
# trn = Transliterator(source='eng', target='hin', build_lookup=True)
# bleu_scores = []
# for i in range(hinglish_sentences.size):

#     transformed_sentence = trn.transform(hinglish_sentences[i])
#     reference = [hindi_true[i].split()]
#     # print(transformed_sentence)
#     transformed_tokens = transformed_sentence.split()
#     bleu_score = sentence_bleu(reference, transformed_tokens)
#     bleu_scores.append(bleu_score)

#     if i % 10 == 0:
#         print("Transliterating sentence ", i+1)
#         print('Original Code mized text :', hinglish_sentences[i])
#         print('Original Native Hindi convertion :', hindi_true[i])
#         print('Transliterated to Hindi :', transformed_sentence)
#         print('Average BLEU score :', np.mean(bleu_scores), '\n')

In [8]:
# average_bleu_score = np.mean(bleu_scores)
# print('Average Bleu Score for Train Set : ', average_bleu_score)

In [9]:
# # The path to the local git repo for Indic NLP library
# INDIC_NLP_LIB_HOME=r"C:\Johns Hopkins\sem1\Machine Translation\MT Final Project\IndicTrans2\indic_nlp_library"

# # The path to the local git repo for Indic NLP Resources
# INDIC_NLP_RESOURCES=r"C:\Johns Hopkins\sem1\Machine Translation\MT Final Project\IndicTrans2\indic_nlp_resources"

# import sys
# sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))

# from indicnlp import common
# common.set_resources_path(INDIC_NLP_RESOURCES)

# from indicnlp import loader
# loader.load()

In [10]:
# Testing using AI4Bharath transliterator
# trn = Transliterator(source='eng', target='hin', build_lookup=True, decode='beamsearch')
# trn = Transliterator(source='eng', target='hin', build_lookup=True)
# ai4b_bleu_scores = []
# lang = 'hi'
# for i in range(hinglish_sentences.size):

  #  transformed_sentence = ItransTransliterator.from_itrans(hinglish_sentences[i],lang)
   # reference = [hindi_true[i].split()]
    # print(transformed_sentence)
    #transformed_tokens = transformed_sentence.split()
    #bleu_score = sentence_bleu(reference, transformed_tokens)
    #ai4b_bleu_scores.append(bleu_score)

    #if i % 10 == 0:
     #   print("Transliterating sentence ", i+1)
      #  print('Original Code mized text :', hinglish_sentences[i])
       # print('Original Native Hindi convertion :', hindi_true[i])
        #print('Transliterated to Hindi :', transformed_sentence)
        #print('Average BLEU score :', np.mean(ai4b_bleu_scores), '\n')

In [11]:
# Testing out Google Transliterate package
from google.transliteration import transliterate_word
def transliterate_all_alphanumeric_parts(sentence, lang_code='hi'):
    transliterated_sentence = []
    words = sentence.split()

    for word in words:
        # Use regex to find all alphanumeric parts in the word
        parts = re.findall(r'[a-zA-Z0-9]+|[^a-zA-Z0-9]+', word)
        
        transliterated_word = ""
        
        for part in parts:
            if part.isalnum():  # Transliterate only alphanumeric parts
                try:
                    # Transliterate the alphanumeric part
                    suggestions = transliterate_word(part, lang_code=lang_code)
                    transliterated_part = suggestions[0] if suggestions else part
                except IndexError:
                    # If transliteration fails, keep the alphanumeric part as is
                    transliterated_part = part
            else:
                # Keep non-alphanumeric parts (like "-" or spaces) as is
                transliterated_part = part
            
            transliterated_word += transliterated_part
        
        transliterated_sentence.append(transliterated_word)

    return ' '.join(transliterated_sentence)

google_bleu_scores = []
google_gleu_scores = []
google_meteor_scores = []
lang = 'hi'
for i in range(hinglish_sentences.size):

    # print(i)
    # transformed_sentence = transliterate_text(hinglish_sentences[i],lang_code = lang)
    transformed_sentence = transliterate_all_alphanumeric_parts(hinglish_sentences[i], lang_code=lang)
    
    reference = [hindi_true[i].split()]
    transformed_tokens = transformed_sentence.split()
    
    bleu_score = sentence_bleu(reference, transformed_tokens)
    gleu_score = sentence_gleu(reference, transformed_tokens)
    met_score = meteor_score(reference, transformed_tokens)
    google_bleu_scores.append(bleu_score)
    google_gleu_scores.append(gleu_score)
    google_meteor_scores.append(met_score)

    if i % 10 == 0:
        print("Transliterating sentence ", i+1)
        print('Original Code mized text :', hinglish_sentences[i])
        print('Original Native Hindi convertion :', hindi_true[i])
        print('Transliterated to Hindi :', transformed_sentence)
        print('Average BLEU score :', np.mean(google_bleu_scores), '\n')
        print('Average GLEU score :', np.mean(google_gleu_scores), '\n')
        print('Average METEOR score :', np.mean(google_meteor_scores), '\n')
        print()

Transliterating sentence  1
Original Code mized text : kya aap in files ko sthayi roop se permanently delete hain

Original Native Hindi convertion : क्या आप इन फ़ाइलों को स्थायी रूप से हटाना चाहते हैं?

Transliterated to Hindi : क्या आप इन फाइल्स को स्थायी रूप से परमानेंटली डिलीट हैं
Average BLEU score : 0.33932513407933634 

Average GLEU score : 0.42105263157894735 

Average METEOR score : 0.6289424860853432 




C:\Users\Nithish\anaconda3\envs\mt_final_project\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\Nithish\anaconda3\envs\mt_final_project\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Transliterating sentence  11
Original Code mized text : aur i chahta hoon ki aap iske bare me soche ki vaisa kyoon hota hai

Original Native Hindi convertion : और मैं चाहता हूँ की आप इसके बारे मे सोचे की वैसा क्यूँ होता है

Transliterated to Hindi : और ी चाहता हूँ की आप इसके बारे में सोचे की वैसा क्यूँ होता है
Average BLEU score : 0.4249095659112498 

Average GLEU score : 0.4914869479113392 

Average METEOR score : 0.6632812074757725 




C:\Users\Nithish\anaconda3\envs\mt_final_project\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Transliterating sentence  21
Original Code mized text : vikipidiya me kaphi tezi se samsamyik vishyon ke bare men lekho ka vikas ho sakta hai jaise ki munbee visphoton ke khabar ka zahir hone ke chnd minton men hi uske bare men angreji vikipidiya men prasngik kariyon ke sath lekh maujood tha.

Original Native Hindi convertion : विकिपिडीया मे काफी तेज़ी से समसामयिक विषयों के बारे में लेखो का विकास हो सकता है जैसे कि मुंबई विस्फोटों के खबर का ज़ाहिर होने के चंद मिनटों में ही उसके बारे में अंग्रेजी विकिपीडिया में प्रासंगिक कड़ियों के साथ लेख मौजूद था।

Transliterated to Hindi : विकिपीडिया में काफी तेज़ी से समसामयिक विषयों के बारे में लेखो का विकास हो सकता है जैसे की मुंबई विस्फोटों के खबर का ज़ाहिर होने के चंद मिनटों में ही उसके बारे में अंग्रेजी विकिपीडिया में प्रासंगिक करियों के साथ लेख मौजूद था.
Average BLEU score : 0.3964619400093708 

Average GLEU score : 0.46615658385045994 

Average METEOR score : 0.6359511377303553 


Transliterating sentence  31
Original Code mized text : ravan ke 

In [13]:
average_bleu_score = np.mean(google_bleu_scores)
average_gleu_score = np.mean(google_gleu_scores)
average_meteor_score = np.mean(google_meteor_scores)
print('Average BLEU Score for Dev Set using Google transliterator: ', average_bleu_score)
print('Average GLEU Score for Dev Set using Google transliterator: ', average_gleu_score)
print('Average METEOR Score for Dev Set using Google transliterator: ', average_meteor_score)
print('Google Transliterate model is very accurate but takes time to transliterate the sentence')

Average BLEU Score for Dev Set using Google transliterator:  0.29570984161501696
Average GLEU Score for Dev Set using Google transliterator:  0.39737549670577893
Average METEOR Score for Dev Set using Google transliterator:  0.5927948483699417
Google Transliterate model is very accurate but takes time to transliterate the sentence
